<a href="https://colab.research.google.com/github/Kartikvenkat98/Heart-Attack-Detection/blob/master/Snorkel_Technique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [360]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [361]:
import os
DRIVE_DIR = '/content/drive/My\ Drive/Colab\ Notebooks/'
os.environ['DRIVE_DIR'] = DRIVE_DIR

In [362]:
cd $DRIVE_DIR

/content/drive/My Drive/Colab Notebooks


In [117]:
!pip install snorkel

In [364]:
import snorkel
import collections

In [365]:
from snorkel import labeling

In [366]:
from snorkel.labeling import labeling_function

In [367]:
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

In [368]:
import pandas as pd
import numpy as np

In [369]:
HEART_ATTACK = 1
NOT_HEARTATTACK = 0

In [ ]:

@labeling_function()
def lf_angles(df):
    count_angles_left = 0
    count_angles_right = 0
    for i in (df['left_arm_angle']):
      if (i > 50 and i < 90):
        count_angles_left += 1
    for i in (df['right_arm_angle']):
      if (i > 50 and i < 90):
        count_angles_right += 1

    if (count_angles_left > 30 or count_angles_right > 30):
      return HEART_ATTACK
    else:
      return NOT_HEARTATTACK


In [ ]:

@labeling_function()
def lf_armratio(df):
    count_ratio_left = 0
    count_ratio_right = 0
    for i in (df['left_arm_n_ratio']):
      #print(i)
      if (i > 0.2 and i < 0.75):
        count_ratio_left += 1
    for i in (df['right_arm_n_ratio']):
      if (i > 0.2 and i < 0.75):
        count_ratio_right += 1

    if (count_ratio_left > 40 or count_ratio_right > 40):
      return HEART_ATTACK
    else:
      return NOT_HEARTATTACK

In [372]:
@labeling_function()
def lf_arm_waistratio(df):
    count_ratio_wleft = 0
    count_ratio_wright = 0
    for i in (df['Left Arm W Ratio']):
      #print(i)
      if (i > 0.4 and i < 1.5):
        count_ratio_wleft += 1
    for i in (df['Right Arm W Ratio']):
      if (i > 0.4 and i < 1.5):
        count_ratio_wright += 1

    if (count_ratio_wleft > 40 or count_ratio_wright > 40):
      return HEART_ATTACK
    else:
      return NOT_HEARTATTACK

In [424]:
df = pd.DataFrame()
path = './videos/videos_1.mp4_clips_json/frames/'
i = 0
for name in os.listdir(path):
    
    path_to_pkl = os.path.join(path, name)
    gf = pd.read_pickle(path_to_pkl)
    f = name.split('.',-1)
    z = f[0]
    df.loc[i,'Filename'] = z
    df.loc[i,'angles'] = lf_angles(gf)
    df.loc[i,'arm_ratio'] = lf_armratio(gf)
    i+=1

In [ ]:
df

In [ ]:
gf = df
gf

In [411]:
kf = pd.read_pickle('./videos/models/video_1.pkl')

In [429]:
mf = pd.concat([gf, model_df], axis = 1)

In [ ]:
mf

In [ ]:
mf = mf.drop(["Filename"],axis=1)
mf

In [432]:
mf.values


array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

## 2) Combining & Cleaning the Labels

Our next step is to apply the labeling functions we wrote to the unlabeled training data.
The result is a *label matrix*, `L_train`, where each row corresponds to a data point and each column corresponds to a labeling function.
Since the labeling functions have unknown accuracies and correlations, their output labels may overlap and conflict.
We use the `LabelModel` to automatically estimate their accuracies and correlations, reweight and combine their labels, and produce our final set of clean, integrated training labels:

In [433]:
#from snorkel.labeling.model import LabelModel
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

# Define the set of labeling functions (LFs)
lfs = [lf_angles, lf_armratio, lf_arm_waistratio]

# Apply the LFs to the unlabeled training data
applier = PandasLFApplier(lfs)
#L_train = applier.apply(df)
L_train = mf.values

# Train the label model and compute the training labels
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=5, log_freq=50, seed=123)
Y = label_model.predict(L=L_train)
#tie_break_policy="abstain"

In [434]:
Y

array([0, 0, 0])

In [419]:
y = list(Y)
y[1]

1

In [ ]:
target_df = pd.read_pickle('./videos/models/target_video_1.pkl')

In [421]:
target_df['Target']

0     0
1     0
2     0
3     0
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    0
12    1
13    1
14    1
15    0
16    0
17    0
Name: Target, dtype: int64

In [ ]:
count_FP_snorkel = 0
for i in range(len(y)):
    if (y[i] == 1 and target_df['Target'][i] == 0):
      count_FP_snorkel += 1

count_FN_snorkel = 0
for i in range(len(y)):
    if (y[i] == 0 and target_df['Target'][i] == 1):
      count_FN_snorkel += 1

count_True_snorkel = 0
for i in range(len(y)):
    if (y[i] == 1 and target_df['Target'][i] == 1) or (y[i] == 0 and target_df['Target'][i] == 0):
      count_True_snorkel += 1

print(count_FP_snorkel)
print(count_FN_snorkel)
count_True_snorkel

In [ ]:
count_FP_model = 0
for i in range(len(mf['Pred'])):
  if (mf['Pred'][i] == 1 and target_df['Target'][i] == 0):
    count_FP_model += 1

count_FN_model = 0
for i in range(len(mf['Pred'])):
  if (mf['Pred'][i] == 0 and target_df['Target'][i] == 1):
    count_FN_model += 1

count_True_model = 0
for i in range(len(mf['Pred'])):
  if (mf['Pred'][i] == 1 and target_df['Target'][i] == 1) or (mf['Pred'][i] == 0 and target_df['Target'][i] == 0):
    count_True_model += 1

print(count_FP_model)
print(count_FN_model)
count_True_model